In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/widsdatathon2020/unlabeled.csv
/kaggle/input/widsdatathon2020/solution_template.csv
/kaggle/input/widsdatathon2020/training_v2.csv
/kaggle/input/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv
/kaggle/input/widsdatathon2020/samplesubmission.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
import catboost as cb
import catboost.utils as cbu
import hyperopt
import sys
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [4]:
from sklearn.utils import class_weight
cw = list(class_weight.compute_class_weight('balanced',
                                             np.unique(train['hospital_death']),
                                             train['hospital_death']))

In [5]:
cw

[0.5472266641208621, 5.793619709412508]

In [6]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178
['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178


In [7]:
'''
def replace_missing_value(df, number_features):
    imputer = SimpleImputer(missing_values=np.nan, strategy="mean")
    df_num = df[number_features]
    imputer.fit(df_num)
    print(df[number_features].head())
    df[number_features] = imputer.transform(df_num)
    return df
'''

'\ndef replace_missing_value(df, number_features):\n    imputer = SimpleImputer(missing_values=np.nan, strategy="mean")\n    df_num = df[number_features]\n    imputer.fit(df_num)\n    print(df[number_features].head())\n    df[number_features] = imputer.transform(df_num)\n    return df\n'

In [8]:
#train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].isna().sum()

In [9]:
'''
train.loc[train['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1
train.loc[train['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1
test.loc[test['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1
test.loc[test['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1
'''


"\ntrain.loc[train['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1\ntrain.loc[train['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1\ntest.loc[test['apache_4a_hospital_death_prob'].isna(), 'apache_4a_hospital_death_prob'] = -1\ntest.loc[test['apache_4a_icu_death_prob'].isna(), 'apache_4a_icu_death_prob'] = -1\n"

In [10]:
#train[train['apache_4a_hospital_death_prob'] ==-1]['apache_4a_hospital_death_prob'].count()

In [11]:
#train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']] = train[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].fillna(-1)
#test[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']] = test[['apache_4a_hospital_death_prob','apache_4a_icu_death_prob']].fillna(-1)

In [12]:
'''
ahd_imp=SimpleImputer(missing_values=-1,strategy="median" )
ahd_imp.fit(train[["apache_4a_hospital_death_prob"]])
train["apache_4a_hospital_death_prob"]=ahd_imp.transform(train[["apache_4a_hospital_death_prob"]])
test["apache_4a_hospital_death_prob"]=ahd_imp.transform(test[["apache_4a_hospital_death_prob"]])
'''

'\nahd_imp=SimpleImputer(missing_values=-1,strategy="median" )\nahd_imp.fit(train[["apache_4a_hospital_death_prob"]])\ntrain["apache_4a_hospital_death_prob"]=ahd_imp.transform(train[["apache_4a_hospital_death_prob"]])\ntest["apache_4a_hospital_death_prob"]=ahd_imp.transform(test[["apache_4a_hospital_death_prob"]])\n'

In [13]:
'''
ahd_imp=SimpleImputer(missing_values=-1,strategy="median" )
ahd_imp.fit(train[["apache_4a_icu_death_prob"]])
train["apache_4a_icu_death_prob"]=ahd_imp.transform(train[["apache_4a_icu_death_prob"]])
test["apache_4a_icu_death_prob"]=ahd_imp.transform(test[["apache_4a_icu_death_prob"]])
'''

'\nahd_imp=SimpleImputer(missing_values=-1,strategy="median" )\nahd_imp.fit(train[["apache_4a_icu_death_prob"]])\ntrain["apache_4a_icu_death_prob"]=ahd_imp.transform(train[["apache_4a_icu_death_prob"]])\ntest["apache_4a_icu_death_prob"]=ahd_imp.transform(test[["apache_4a_icu_death_prob"]])\n'

In [14]:
'apache_4a_hospital_death_prob' in train_numerical_cols

True

In [15]:
cols_to_remove = [
    'apache_post_operative','d1_albumin_max','d1_albumin_min','d1_bilirubin_max','d1_bilirubin_min','d1_bun_max','d1_bun_min','d1_creatinine_max',
    'd1_creatinine_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_glucose_max','d1_hemaglobin_min','d1_hematocrit_max','d1_hematocrit_min',
    'd1_mbp_noninvasive_max','d1_mbp_noninvasive_min','d1_platelets_min','d1_sodium_min','d1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','d1_wbc_max','d1_wbc_min',
    'h1_albumin_min','h1_arterial_pco2_max','h1_arterial_pco2_min','h1_arterial_ph_min','h1_bilirubin_max','h1_bilirubin_min','h1_bun_max','h1_bun_min','h1_calcium_min',
    'h1_creatinine_max','h1_creatinine_min', 'h1_diasbp_noninvasive_max','h1_diasbp_noninvasive_min','h1_glucose_min', 'h1_hco3_max', 'h1_hco3_min', 'h1_hemaglobin_max',
    'h1_hemaglobin_min', 'h1_hematocrit_max', 'h1_hematocrit_min', 'h1_inr_max', 'h1_inr_min','h1_lactate_max','h1_lactate_min','h1_mbp_noninvasive_max','h1_mbp_noninvasive_min',
    'h1_pao2fio2ratio_min','h1_platelets_max','h1_platelets_min','h1_potassium_min','h1_sodium_min', 'h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min','h1_wbc_max','h1_wbc_min',
    'paco2_for_ph_apache','readmission_status'
    ]

'''
cols_to_remove = [
    'd1_albumin_max', 'd1_bilirubin_max', 'd1_bilirubin_min', 'd1_bun_max','d1_bun_min','d1_creatinine_max',
    'd1_creatinine_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min', 'd1_hematocrit_max', 'd1_mbp_noninvasive_max','d1_mbp_noninvasive_min', 'd1_platelets_min',
    'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','d1_wbc_max', 'h1_albumin_min', 'h1_arterial_pco2_min','h1_bilirubin_max','h1_bilirubin_min','h1_bun_min',
    'h1_calcium_min',
    'h1_creatinine_max','h1_creatinine_min', 'h1_diasbp_noninvasive_max','h1_diasbp_noninvasive_min','h1_glucose_min', 'h1_hco3_min', 'h1_hemaglobin_min', 'h1_hematocrit_min',
    'h1_inr_max', 'h1_inr_min', 'h1_lactate_min', 'h1_mbp_noninvasive_max','h1_mbp_noninvasive_min', 'h1_platelets_min', 'h1_sodium_min', 
    'h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min', 'h1_wbc_min', 'paco2_for_ph_apache','readmission_status'
]
'''
cols_to_remove = [
    'h1_inr_max', 'h1_inr_min', 'paco2_for_ph_apache','readmission_status'
]

In [16]:
len(cols_to_remove)

4

In [17]:
np.setdiff1d(cols_to_remove, train_numerical_cols)

array([], dtype='<U19')

In [18]:
np.setdiff1d(cols_to_remove, train_categorical_cols)

array(['h1_inr_max', 'h1_inr_min', 'paco2_for_ph_apache',
       'readmission_status'], dtype='<U19')

In [19]:
train[['encounter_id','apache_4a_hospital_death_prob']].head(5)

,encounter_id,apache_4a_hospital_death_prob
0,66154,0.10
1,114252,0.47
2,119783,0.00
3,79267,0.04
4,92056,NaN


In [20]:
train['gcs'] = train[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)
test['gcs'] = test[['gcs_eyes_apache','gcs_motor_apache','gcs_unable_apache','gcs_verbal_apache']].mean(axis =1)

In [21]:
train['gcs'].fillna(train['gcs'].mean(), inplace = True)
test['gcs'].fillna(test['gcs'].mean(), inplace = True)

In [22]:
#train = replace_missing_value(train, train_numerical_cols)
#train[test_numerical_cols].apply(lambda x: x.fillna(x.mean()),axis=0)

In [23]:
#test[test_numerical_cols].fillna(test[test_numerical_cols].mean(), inplace=True)
#test[test_numerical_cols].apply(lambda x: x.fillna(x.mean()),axis=0)

In [24]:
#test[test_numerical_cols].isna().any()

In [25]:
train.drop(train.loc[train['hospital_admit_source']=='Observation'].index, inplace=True)
#train.drop(train.loc[train['gender'].isna()].index, inplace=True)

In [26]:
'''
for col in train_categorical_cols:
    print('\n' + col)
    print(train[col].unique())
    print(test[col].unique())
'''

"\nfor col in train_categorical_cols:\n    print('\n' + col)\n    print(train[col].unique())\n    print(test[col].unique())\n"

In [27]:
train.shape

(91703, 187)

In [28]:
100 - (7907 / 91678) * 100

91.37524815113768

In [29]:
#train.loc[train['hospital_death']== 1].count()
#91678  - 7907
83771 / 7907

10.594536486657391

In [30]:
#train[train['apache_2_bodysystem'] == 'Undefined Diagnoses'].count()

In [31]:
train['hospital_admit_source'].unique()

array(['Floor', 'Emergency Department', 'Operating Room', nan,
       'Direct Admit', 'Other Hospital', 'Other ICU', 'ICU to SDU',
       'Recovery Room', 'Chest Pain Center', 'Step-Down Unit (SDU)',
       'Acute Care/Floor', 'PACU', 'ICU', 'Other'], dtype=object)

In [32]:
#train[train['apache_2_bodysystem'] == 'Undefined diagnoses'].count()
train.loc[train['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'
test.loc[test['apache_2_bodysystem'] == 'Undefined Diagnoses', 'apache_2_bodysystem'] = 'Undefined diagnoses'

In [33]:
train['apache_2_bodysystem'] = train['apache_2_bodysystem'].fillna("Undefined diagnoses")
test['apache_2_bodysystem'] = test['apache_2_bodysystem'].fillna("Undefined diagnoses")

In [34]:
train['ethnicity'] = train['ethnicity'].fillna("Other/Unknown")
test['ethnicity'] = test['ethnicity'].fillna("Other/Unknown")

In [35]:
train['hospital_admit_source'] = train['hospital_admit_source'].fillna("Other")
test['hospital_admit_source'] = test['hospital_admit_source'].fillna("Other")

In [36]:
train['icu_admit_source'] = train['icu_admit_source'].fillna("Other ICU")
test['icu_admit_source'] = test['icu_admit_source'].fillna("Other ICU")

In [37]:
train['gender'] = train['gender'].fillna("NA")
test['gender'] = test['gender'].fillna("NA")

In [38]:
train[train['apache_3j_bodysystem'].isna()].head(3)

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,gcs
44,71147,52594,118,0,60.0,35.452865,1,Caucasian,M,170.2,Operating Room,Operating Room / Recovery,85,admit,SICU,0.002778,0,102.7,NaN,NaN,0.25,0,0.0,NaN,6.0,0.83,NaN,4.0,6.0,0.0,5.0,254.0,97.0,42.0,0.0,145.0,NaN,NaN,NaN,NaN,34.0,137.0,35.7,NaN,1.0,14.4,109.0,48.0,119.0,55.0,119.0,55.0,96.0,71.0,122.0,75.0,145.0,77.0,145.0,77.0,34.0,8.0,100.0,91.0,195.0,121.0,187.0,124.0,187.0,124.0,37.0,35.7,109.0,56.0,119.0,68.0,119.0,68.0,76.0,71.0,122.0,94.0,145.0,104.0,145.0,104.0,27.0,12.0,100.0,92.0,187.0,160.0,183.0,152.0,183.0,152.0,35.7,35.7,NaN,NaN,NaN,NaN,6.0,6.0,8.9,8.6,0.85,0.83,254.0,111.0,28.0,27.0,14.0,13.9,42.3,42.0,NaN,NaN,NaN,NaN,208.0,198.0,4.3,4.2,138.0,137.0,14.4,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,Undefined diagnoses,3.75
83,63576,36677,118,0,56.0,24.542930,1,His

In [39]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("NA")
test[test_categorical_cols] = test[test_categorical_cols].fillna("NA")

In [40]:
train[train_numerical_cols] = train[train_numerical_cols].fillna(train[train_numerical_cols].median())
test[test_numerical_cols] = test[test_numerical_cols].fillna(test[test_numerical_cols].median())


In [41]:
#train[train_numerical_cols].isna().any()

In [42]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()
allcols = train.columns.values.tolist()
inputcols = allcols[4:]
outputcols = allcols[3]

In [43]:
inputcols.remove('gcs_eyes_apache')
inputcols.remove('gcs_motor_apache')
inputcols.remove('gcs_unable_apache')
inputcols.remove('gcs_verbal_apache')

In [44]:
len(inputcols)

179

In [45]:
'''
cols_to_remove.append('h1_inr_max')
cols_to_remove.append('h1_inr_min')
cols_to_remove.append('paco2_for_ph_apache')
cols_to_remove.append('d1_bilirubin_max')
cols_to_remove.append('d1_bilirubin_min')
'''

"\ncols_to_remove.append('h1_inr_max')\ncols_to_remove.append('h1_inr_min')\ncols_to_remove.append('paco2_for_ph_apache')\ncols_to_remove.append('d1_bilirubin_max')\ncols_to_remove.append('d1_bilirubin_min')\n"

In [46]:
'''
for col in cols_to_remove:
    inputcols.remove(col)
'''

'\nfor col in cols_to_remove:\n    inputcols.remove(col)\n'

In [47]:
len(inputcols)

179

In [48]:
#train_categorical_cols.append('hospital_id')
#inputcols.append('hospital_id')

In [49]:
'hospital_id' in inputcols

False

In [50]:
X = train[inputcols]
y = train[outputcols]

In [51]:
train_pool = Pool(data=X,label = y,cat_features=train_categorical_cols,
#                   baseline= X_train[""], ## 
                   #group_id = ['encounter_id']
                 )

In [52]:
### hyperparameter tuning example grid for catboost : 
'''
grid = {'learning_rate': [0.01, 0.04],
        'depth': [10],
        #'l2_leaf_reg': [3,4,5],
        "iterations": [500],
        "custom_metric":['Logloss', 'AUC']}

model = CatBoostClassifier(task_type="GPU")

## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`
grid_search_result = model.grid_search(grid, 
                                       train_pool,
                                       plot=True,
                                       refit = True, #  refit best model on all data
                                      partition_random_seed=42)

print(model.get_best_score())
print("best model params: \n",grid_search_result["params"])
'''

'\ngrid = {\'learning_rate\': [0.01, 0.04],\n        \'depth\': [10],\n        #\'l2_leaf_reg\': [3,4,5],\n        "iterations": [500],\n        "custom_metric":[\'Logloss\', \'AUC\']}\n\nmodel = CatBoostClassifier(task_type="GPU")\n\n## can also do randomized search - more efficient typically, especially for large search space - `randomized_search`\ngrid_search_result = model.grid_search(grid, \n                                       train_pool,\n                                       plot=True,\n                                       refit = True, #  refit best model on all data\n                                      partition_random_seed=42)\n\nprint(model.get_best_score())\nprint("best model params: \n",grid_search_result["params"])\n'

In [53]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08000100126274019, depth = 10,loss_function='Logloss', eval_metric = 'AUC', 
                             custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0901.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

'\npredictions = []\nscale_pos_weight = 10.594536486657391\n\n#0.07260908767\nfor i in range(10):\n    clf = CatBoostClassifier(learning_rate=0.08000100126274019, depth = 10,loss_function=\'Logloss\', eval_metric = \'AUC\', \n                             custom_metric= [\'AUC\'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level=\'Silent\')\n    clf.fit(train_pool, plot=True,silent=True)\n    predictions.append(clf.predict_proba(test[inputcols])[:,1])\n    print(clf.get_best_score())\n    \nprediction = np.mean(predictions, axis=0)\ntest["hospital_death"] = prediction\ntest[["encounter_id","hospital_death"]].to_csv("0901.csv",index=False)\n#{\'learn\': {\'Logloss\': 0.08512141544067534}} # 08177070126274019 #08000100126274019\n'

In [54]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08000100126274019, boosting_type='Ordered', depth = 10,loss_function='Logloss', eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0902.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}} # 08177070126274019 #08000100126274019
'''

'\npredictions = []\nscale_pos_weight = 10.594536486657391\n\n#0.07260908767\nfor i in range(10):\n    clf = CatBoostClassifier(learning_rate=0.08000100126274019, boosting_type=\'Ordered\', depth = 10,loss_function=\'Logloss\', eval_metric = \'AUC\', custom_metric= [\'AUC\'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level=\'Silent\')\n    clf.fit(train_pool, plot=True,silent=True)\n    predictions.append(clf.predict_proba(test[inputcols])[:,1])\n    print(clf.get_best_score())\n    \nprediction = np.mean(predictions, axis=0)\ntest["hospital_death"] = prediction\ntest[["encounter_id","hospital_death"]].to_csv("0902.csv",index=False)\n#{\'learn\': {\'Logloss\': 0.08512141544067534}} # 08177070126274019 #08000100126274019\n'

In [55]:
'''
predictions = []
scale_pos_weight = 10.594536486657391

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function='Logloss', eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0217.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

'\npredictions = []\nscale_pos_weight = 10.594536486657391\n\n#0.07260908767\nfor i in range(10):\n    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function=\'Logloss\', eval_metric = \'AUC\', custom_metric= [\'AUC\'], iterations=1000, l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level=\'Silent\')\n    clf.fit(train_pool, plot=True,silent=True)\n    predictions.append(clf.predict_proba(test[inputcols])[:,1])\n    print(clf.get_best_score())\n    \nprediction = np.mean(predictions, axis=0)\ntest["hospital_death"] = prediction\ntest[["encounter_id","hospital_death"]].to_csv("0217.csv",index=False)\n#{\'learn\': {\'Logloss\': 0.08512141544067534}}\n'

In [56]:

class UciAdultClassifierObjective(object):
    def __init__(self, dataset, const_params, fold_count):
        self._dataset = dataset
        self._const_params = const_params.copy()
        self._fold_count = fold_count
        self._evaluated_count = 0
        
    def _to_catboost_params(self, hyper_params):
        return {
            'learning_rate': hyper_params['learning_rate'],
            'depth': hyper_params['depth'],
            'l2_leaf_reg': hyper_params['l2_leaf_reg']}
    
    # hyperopt optimizes an objective using `__call__` method (e.g. by doing 
    # `foo(hyper_params)`), so we provide one
    def __call__(self, hyper_params):
        # join hyper-parameters provided by hyperopt with hyper-parameters 
        # provided by the user
        params = self._to_catboost_params(hyper_params)
        params.update(self._const_params)
        
        print('evaluating params={}'.format(params), file=sys.stdout)
        sys.stdout.flush()
        
        # we use cross-validation for objective evaluation, to avoid overfitting
        scores = cb.cv(
            pool=self._dataset,
            params=params,
            fold_count=self._fold_count,
            partition_random_seed=20181224,
            verbose=False)
        
        # scores returns a dictionary with mean and std (per-fold) of metric 
        # value for each cv iteration, we choose minimal value of objective 
        # mean (though it will be better to choose minimal value among all folds)
        # because noise is additive
        max_mean_auc = np.max(scores['test-AUC-mean'])
        print('evaluated score={}'.format(max_mean_auc), file=sys.stdout)
        
        self._evaluated_count += 1
        print('evaluated {} times'.format(self._evaluated_count), file=sys.stdout)
        
        # negate because hyperopt minimizes the objective
        return {'loss': -max_mean_auc, 'status': hyperopt.STATUS_OK}
    
def find_best_hyper_params(dataset, const_params, max_evals=100):    
    # we are going to optimize these three parameters, though there are a lot more of them (see CatBoost docs)
    parameter_space = {
        'learning_rate': hyperopt.hp.uniform('learning_rate', 0.01, 0.1),
        'depth': hyperopt.hp.randint('depth', 12),
        'l2_leaf_reg': hyperopt.hp.uniform('l2_leaf_reg', 1, 10)}
    objective = UciAdultClassifierObjective(dataset=dataset, const_params=const_params, fold_count=6)
    trials = hyperopt.Trials()
    best = hyperopt.fmin(
        fn=objective,
        space=parameter_space,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        rstate=np.random.RandomState(seed=20181224))
    return best

def train_best_model(X, y, const_params, max_evals=100, use_default=False):
    # convert pandas.DataFrame to catboost.Pool to avoid converting it on each 
    # iteration of hyper-parameters optimization
    dataset = Pool(X, y, cat_features=train_categorical_cols)
    
    if use_default:
        # pretrained optimal parameters
        best = {
            'learning_rate': 0.4234185321620083, 
            'depth': 5, 
            'l2_leaf_reg': 9.464266235679002}
    else:
        best = find_best_hyper_params(dataset, const_params, max_evals=max_evals)
    
    # merge subset of hyper-parameters provided by hyperopt with hyper-parameters 
    # provided by the user
    hyper_params = best.copy()
    hyper_params.update(const_params)
    
    # drop `use_best_model` because we are going to use entire dataset for 
    # training of the final model
    hyper_params.pop('use_best_model', None)
    
    model = cb.CatBoostClassifier(**hyper_params)
    model.fit(dataset, verbose=False)
    
    return model, hyper_params


# make it True if your want to use GPU for training
have_gpu = True
# skip hyper-parameter optimization and just use provided optimal parameters
use_optimal_pretrained_params = False
# number of iterations of hyper-parameter search
hyperopt_iterations = 15

const_params = dict({
    'task_type': 'GPU' if have_gpu else 'CPU',
    'loss_function': 'Logloss',
    'eval_metric': 'AUC', 
    'custom_metric': ['AUC'],
    'iterations': 1000,
    'random_seed': 20181224})

model, params = train_best_model(
    X, y, 
    const_params, 
    max_evals=hyperopt_iterations, 
    use_default=use_optimal_pretrained_params)
print('best params are {}'.format(params), file=sys.stdout)

evaluating params={'learning_rate': 0.08166369766256534, 'depth': 8, 'l2_leaf_reg': 8.835837790286535, 'task_type': 'GPU', 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': ['AUC'], 'iterations': 1000, 'random_seed': 20181224}
evaluated score=0.9034430533647537
evaluated 1 times
evaluating params={'learning_rate': 0.08216426010912047, 'depth': 3, 'l2_leaf_reg': 3.841780975474402, 'task_type': 'GPU', 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': ['AUC'], 'iterations': 1000, 'random_seed': 20181224}
evaluated score=0.9010914315780004
evaluated 2 times
evaluating params={'learning_rate': 0.03513458486822593, 'depth': 5, 'l2_leaf_reg': 9.464266235679002, 'task_type': 'GPU', 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'custom_metric': ['AUC'], 'iterations': 1000, 'random_seed': 20181224}
evaluated score=0.9012968391180038
evaluated 3 times
evaluating params={'learning_rate': 0.01278048883653391, 'depth': 3, 'l2_leaf_reg': 7.1682035345674, 'task_ty

In [57]:
print('hyper completed')

hyper completed


In [58]:
'''
predictions = []
scale_pos_weight = 10.594536486657391
random_seed = 20181224

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08577070126274019,depth = 10,loss_function='Logloss', 
                             eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, 
                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0212.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

'\npredictions = []\nscale_pos_weight = 10.594536486657391\nrandom_seed = 20181224\n\n#0.07260908767\nfor i in range(10):\n    clf = CatBoostClassifier(learning_rate=0.08577070126274019,depth = 10,loss_function=\'Logloss\', \n                             eval_metric = \'AUC\', custom_metric= [\'AUC\'], iterations=1000, \n                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level=\'Silent\')\n    clf.fit(train_pool, plot=True,silent=True)\n    predictions.append(clf.predict_proba(test[inputcols])[:,1])\n    print(clf.get_best_score())\n    \nprediction = np.mean(predictions, axis=0)\ntest["hospital_death"] = prediction\ntest[["encounter_id","hospital_death"]].to_csv("0212.csv",index=False)\n#{\'learn\': {\'Logloss\': 0.08512141544067534}}\n'

In [59]:
'''
predictions = []
scale_pos_weight = 10.594536486657391
random_seed = 20181224

#0.07260908767
for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function='Logloss', 
                             eval_metric = 'AUC', custom_metric= ['AUC'], iterations=1000, 
                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level='Silent')
    clf.fit(train_pool, plot=True,silent=True)
    predictions.append(clf.predict_proba(test[inputcols])[:,1])
    print(clf.get_best_score())
    
prediction = np.mean(predictions, axis=0)
test["hospital_death"] = prediction
test[["encounter_id","hospital_death"]].to_csv("0210.csv",index=False)
#{'learn': {'Logloss': 0.08512141544067534}}
'''

'\npredictions = []\nscale_pos_weight = 10.594536486657391\nrandom_seed = 20181224\n\n#0.07260908767\nfor i in range(10):\n    clf = CatBoostClassifier(learning_rate=0.08277070126274019,depth = 10,loss_function=\'Logloss\', \n                             eval_metric = \'AUC\', custom_metric= [\'AUC\'], iterations=1000, \n                             l2_leaf_reg = 4,task_type="GPU", random_seed=i, logging_level=\'Silent\')\n    clf.fit(train_pool, plot=True,silent=True)\n    predictions.append(clf.predict_proba(test[inputcols])[:,1])\n    print(clf.get_best_score())\n    \nprediction = np.mean(predictions, axis=0)\ntest["hospital_death"] = prediction\ntest[["encounter_id","hospital_death"]].to_csv("0210.csv",index=False)\n#{\'learn\': {\'Logloss\': 0.08512141544067534}}\n'

In [60]:
print('completed')

completed
